In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Cow killer + bone burrier

bones_inv = load_image('bones_inv.png')
bury_bones = True

target()
total_cows = 0
logins = 0
start_time = mark_time()
last_fight = mark_time()
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
    if mark_time() - last_fight > 10*60:
        raise RuntimeError('No fights in a while, not good.')
            
    mainscreen = get_mainscreen()
    hp_r = find_colors([255,0,0],mainscreen,tol=0.01,mode='hsl') - [msxc-msxs,msyc-msys]
    hp_g = find_colors([0,255,0],mainscreen,tol=0.01,mode='hsl') - [msxc-msxs,msyc-msys]
    
    if len(filter_radius(hp_r,[0,0],100)) > 10: #in combat
        sleep(0.2)
        last_fight = mark_time()
        continue
        
    if bury_bones:
        a = find_colors([158,33,17],mainscreen,tol=0.05,mode='hsl')
        c = find_colors([225,225,225],mainscreen,tol=0.1,mode='hsl')
        bones = filter_near(a,c,10)
        if len(bones) > 0:
            print('collecting bones...')
            bones = closest_exp([msw/2,msh/2],bones,100,N=5)
            for bone in bones:
                move_mouse(*(bone+[msxs,msys]))
                sleep(0.2)
                uptext = get_uptext()
                orange = find_colors([225,128,50],uptext,mode='hsl',tol=(0.03,0.2,0.1))
                print('bones:',len(orange))
                if len(orange) > 50:
                    click_mouse(*(bone+[msxs,msys]))
                    flag_wait(init=1.0,step=0.2,post=0.5,imax=5)
                    break
            
    if bury_bones and count_inv() > 25: #time to bury
        print('burying bones...')
        for i in range(60):
            inventory = get_inventory()
            bury = find_best_bitmap(bones_inv,inventory,tol=0.35,mode='xcorr')
            if len(bury) > 0:
                np.random.shuffle(bury)
                click_mouse(*(bury[0]+[ivxs,ivys]+np.asarray(bones_inv.shape[:2])/2))
                sleep(0.1)
            else:
                break
        drop_all()
        continue
    
    mainscreen = get_mainscreen()
    a = find_colors([122,89,43],mainscreen,tol=(0.05,0.2,0.05),mode='hsl')
    b = find_colors([64,46,23],mainscreen,tol=(0.05,0.2,0.05),mode='hsl')
    c = find_colors([200,200,200],mainscreen,tol=(0.2,0.1,0.1),mode='hsl')
    cow_white = filter_near(filter_near(c,b,50),a,50)
    a = filter_near(a,cow_white,50)
    b = filter_near(b,cow_white,50)
    if len(a) == 0 and len(b) == 0:
        continue
    elif len(a) == 0:
        cows = b
    elif len(b) == 0:
        cows = a
    else:
        cows = np.concatenate([a,b])
        
    if len(cows) > 0:
        #np.random.shuffle(cows)
                
        #dist = np.sqrt(np.sum(np.square(cows-[msw/2,msh/2]),axis=1))
        #dist += 50*(2.0*np.random.random(dist.shape)-1.0)
        #sorter = np.argsort(dist)
        
        cows = closest_exp([msw/2,msh/2],cows,100,N=5)
        for cow in cows:
            move_mouse(*(cow+[msxs,msys]))
            sleep(0.1)
            uptext = get_uptext()
            greentxt = find_colors([0,225,0],uptext,tol=0.05,mode='hsl')
            if len(greentxt) > 10:
                total_cows += 1
                print('%0.2f seconds per cow'%((mark_time()-start_time)/total_cows))
                print('Attacking cow %i'%total_cows)
                click_mouse(*(cow+[msxs,msys]))
                flag_wait(init=1.0,step=0.2,post=0.5,imax=5)
                sleep(3.0)
                break